In [28]:
import os
import boto3

In [6]:
%pwd

'f:\\DS-Projects\\Tom-Jerry_Classification\\research'

In [7]:
os.chdir("../")

In [8]:
%pwd

'f:\\DS-Projects\\Tom-Jerry_Classification'

In [1]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    aws_access_key: str
    aws_secret_key: str
    bucket_name: str
    bucket_filename: str
    local_data_file: str
    unzip_dir: Path

In [30]:
from CNNClassifier.constant import CONFIG_FILE_PATH , PARAMS_FILE_PATH , AWS_ACCESS_KEY , AWS_SECRET_KEY
from CNNClassifier.utils.common import read_yaml , create_directories

In [32]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_directory])
        

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_directory,
            aws_access_key = AWS_ACCESS_KEY,
            aws_secret_key = AWS_SECRET_KEY,
            bucket_name= config.bucket_name,
            bucket_filename= config.bucket_file_name,
            # source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      
    

In [33]:
import os
# import urllib.request as request
from CNNClassifier import logger
import zipfile
from CNNClassifier.utils.common import get_size

In [34]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            s3_client = boto3.client('s3', aws_access_key_id= self.config.aws_access_key,
                         aws_secret_access_key=self.config.aws_secret_key)
            s3_client.download_file(self.config.bucket_name, self.config.bucket_filename,  self.config.local_data_file)
            logger.info(f"{self.config.local_data_file} download from S3 ! with following bucket: {self.config.bucket_name}")            
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

            
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path , exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info("Extracting the Zip Files and stores inside the Artifcats Folder")
        

In [35]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

2023-06-10 23:14:19,294: 			 INFO:	 common:	 yaml file: config\config.yaml loaded successfully 

2023-06-10 23:14:19,297: 			 INFO:	 common:	 yaml file: params.yaml loaded successfully 

2023-06-10 23:14:19,301: 			 INFO:	 common:	 created directory at: artifacts 

2023-06-10 23:14:19,305: 			 INFO:	 common:	 created directory at: artifacts/data_ingestion 



2023-06-10 23:14:29,237: 			 INFO:	 1806091137:	 artifacts/data_ingestion/DataSet.zip download from S3 ! with following bucket: cnndataset1 

